# Toronto Neighborhoods - part 1

In [1]:
#pip install geocoder

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
# import geocoder
# from geopy.geocoders import Nominatim

Scraping

In [3]:
wikipage = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(wikipage.content,'lxml')
table = soup.find_all('table')[0]
table = pd.read_html(str(table))

Transform into pandas dataframe

In [4]:
df=pd.DataFrame(table[0])

I keep only the rows with a Borough indicated

In [5]:
df = df[df["Borough"]!="Not assigned"]
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Checking for "Not assigned" Neighborhoos

In [6]:
len(df[df["Neighborhood"]=="Not assigned"])

0

In [7]:
df = df.reset_index(drop=True)

## Adding coordinates

I tried the following code, but I get no responde from geocoder

In [8]:
# latitude = []
# longitude = []

# for postal_code in df["Postal Code"]:
#     # initialize your variable to None
#     lat_lng_coords = None
    
#     c=0

#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
#         print(".", end='')
        
#     print(c,postal_code,"OK")
#     c=c+1

#     latitude.append(lat_lng_coords[0])
#     longitude.append(lat_lng_coords[1])

# df["Latitude"] = latitude
# df["Longitude"] = longitude

# df.head()

Tried this other method but also failed

In [9]:
# latitude = []
# longitude = []
# geolocator = Nominatim(user_agent="toronto_explorer")

# for postal_code in df["Postal Code"]:
#     location = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code))
#     if location == None:
#         latitude.append(None)
#         longitude.append(None)
#     else:
#         latitude.append(location.latitude)
#         longitude.append(location.longitude)

#     print(postal_code,"OK")
        
# df["Latitude"] = latitude
# df["Longitude"] = longitude

# df.head()

Changing to csv mthod

In [10]:
#!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

In [11]:
geodata = pd.read_csv('Geospatial_Coordinates.csv')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
geodata.shape

(103, 3)

In [13]:
df=pd.merge(df,geodata,on="Postal Code")
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
df.shape

(103, 5)

## Clustering

In [15]:
import numpy as np # library to handle data in a vectorized manner

import requests # library to handle requests

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium

In [16]:
latitude = 43.7171819
longitude = -79.4057996

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# DELETE CREDENTIALS

In [17]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
OAUTH_TOKEN = ''
VERSION = '20180604'

print('Your credentails:')

Your credentails:


In [39]:
radius = 5000
#LIMIT = 100

In [40]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [43]:
print(toronto_venues.shape)
toronto_venues.head()

(10266, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Island Foods,43.745866,-79.346035,Caribbean Restaurant
3,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518,Supermarket
4,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space


In [44]:
nuevo = toronto_venues.groupby('Neighborhood').count()

In [45]:
nuevo.shape

(99, 6)

In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Art Gallery,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,Agincourt,0.0,0.0,0.00,0.0,0.01,0.0,0.0,0.0,0.01,...,0.010000,0.00,0.020000,0.00,0.0,0.00,0.0,0.01,0.00,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.01,...,0.000000,0.01,0.020000,0.01,0.0,0.02,0.0,0.00,0.02,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.00,...,0.000000,0.00,0.010000,0.01,0.0,0.00,0.0,0.00,0.00,0.0
3,Bayview Village,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.010309,0.00,0.010309,0.00,0.0,0.00,0.0,0.00,0.00,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.00,...,0.000000,0.00,0.000000,0.01,0.0,0.00,0.0,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"Willowdale, Willowdale West",0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.010000,0.00,0.010000,0.00,0.0,0.01,0.0,0.00,0.00,0.0
95,Woburn,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.000000,0.00,0.000000,0.00,0.0,0.01,0.0,0.01,0.00,0.0
96,Woodbine Heights,0.0,0.0,0.00,0.0,0.02,0.0,0.0,0.0,0.00,...,0.010000,0.00,0.000000,0.01,0.0,0.00,0.0,0.00,0.00,0.0
97,York Mills West,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.000000,0.00,0.000000,0.00,0.0,0.00,0.0,0.00,0.00,0.0


In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Agincourt,Chinese Restaurant,Coffee Shop,Bakery,Caribbean Restaurant,Pizza Place,Park,Sushi Restaurant,Noodle House,Falafel Restaurant,Greek Restaurant,Indian Restaurant,Japanese Restaurant,Fast Food Restaurant,Supermarket,Sandwich Place
1,"Alderwood, Long Branch",Coffee Shop,Café,Burger Joint,Bakery,Breakfast Spot,Burrito Place,Pizza Place,Park,Seafood Restaurant,Grocery Store,Ice Cream Shop,Sporting Goods Shop,Middle Eastern Restaurant,Yoga Studio,Fast Food Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Restaurant,Grocery Store,Middle Eastern Restaurant,Bakery,Bagel Shop,Park,Clothing Store,Bubble Tea Shop,Deli / Bodega,Liquor Store,Sushi Restaurant,Japanese Restaurant,Café,Jewelry Store
3,Bayview Village,Coffee Shop,Middle Eastern Restaurant,Supermarket,Furniture / Home Store,Bakery,Japanese Restaurant,Thai Restaurant,Bagel Shop,Bank,Hotel,Restaurant,Burger Joint,Sushi Restaurant,Ice Cream Shop,Bubble Tea Shop
4,"Bedford Park, Lawrence Manor East",Café,Grocery Store,Park,Bakery,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Tea Room,Trail,Liquor Store,Supermarket,Thai Restaurant,Steakhouse,Brewery


In [50]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 1, 2, 4, 4, 4, 2])

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Middle Eastern Restaurant,Caribbean Restaurant,Supermarket,Coffee Shop,...,Bakery,Liquor Store,Japanese Restaurant,Burger Joint,Grocery Store,Mediterranean Restaurant,Gym / Fitness Center,American Restaurant,Restaurant,Burrito Place
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Park,Supermarket,Coffee Shop,Burger Joint,...,Middle Eastern Restaurant,Filipino Restaurant,Steakhouse,Burrito Place,Furniture / Home Store,Café,Seafood Restaurant,Indian Restaurant,Chinese Restaurant,BBQ Joint
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Restaurant,Café,...,Plaza,Historic Site,Mediterranean Restaurant,Liquor Store,Farmers Market,Brewery,Japanese Restaurant,Hotel,Bakery,Dance Studio
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Coffee Shop,Italian Restaurant,Café,Vietnamese Restaurant,...,Bakery,Burger Joint,Pizza Place,Bagel Shop,Chinese Restaurant,Mediterranean Restaurant,Liquor Store,Fast Food Restaurant,Sushi Restaurant,Trail
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Park,Bakery,Dessert Shop,...,Café,Mexican Restaurant,Spa,Restaurant,Japanese Restaurant,Bookstore,Plaza,Supermarket,Farmers Market,Dance Studio


## problema con int en cluster label

In [52]:
toronto_merged.shape

(103, 21)

In [53]:
toronto_merged[toronto_merged['Cluster Labels'].isnull()]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue


In [55]:
# toronto_merged = toronto_merged[toronto_merged['Cluster Labels'].notnull()]
# toronto_merged.shape

In [56]:
# toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [57]:
# toronto_merged.head()

In [58]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [59]:
nuevo = toronto_merged.groupby('Cluster Labels').count()

In [60]:
nuevo

,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,,,,,,
0,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
1,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24
2,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19
3,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32
4,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
